Run this notebook before anything else!

In [ ]:
import os
import requests
import time
from p4tools import io # from https://github.com/michaelaye/p4tools : 


In [ ]:

#before starting: update base folder name to an absolute path on your system, or do not change any paths
base_folder = '../../'

#set up folders
DataSubFolders=['ClusteringResults','Figures','Images/HiRISE','Images/HiRISE_8bit_and_P4_mask','Models','SummaryResults']
for f in DataSubFolders:
    if not os.path.exists(base_folder+'Data/'+f):
        os.makedirs(base_folder+'Data/'+f)

#HiRISE data URL and save path
URL_base = 'https://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/RDR/ESP/ORB_0'
data_folder = base_folder+'Data/Images/HiRISE/' #recommended path

In [ ]:

def DownloadHiRISE(URL_base,data_folder):
    metadata_df = io.get_meta_data()
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)     
    for index, row in metadata_df.iterrows():
        t0 = time.time()
        folder= str(int(int(row['OBSERVATION_ID'][4:10])/100)*100)+'_0'+str(int(int(row['OBSERVATION_ID'][4:10])/100)*100+99)
        FileName = row['OBSERVATION_ID']+'_RGB.NOMAP.JP2'
                        
        if not os.path.isfile(os.path.join(data_folder, FileName)):
            #have not lready downloaded file, so now download
            Full_URL = URL_base+folder+'/'+row['OBSERVATION_ID']+'/'+FileName
            r = requests.head(Full_URL)
            if r.status_code == requests.codes.ok:
                #file exists at URL
                print('Downloading and saving HiRISE image ',str(index),' of ',str(metadata_df.shape[0]),'from: ',Full_URL)
                r = requests.get(Full_URL)
                with open(os.path.join(data_folder, FileName), 'wb') as f:
                    f.write(r.content)
        else:
            print('HiRISE file already exists at: ',os.path.join(data_folder, FileName))    
        t1 = time.time()
        print("time for this image = ",t1-t0)

In [ ]:
t0 = time.time()
DownloadHiRISE(URL_base,data_folder)#total size of 221 JP2 files is 39.3 GB on disk
t1 = time.time()
print("time to download all images = ",t1-t0) #takes around an hour
